# PyBullet을 이용한 진자(Pendulum) 강화학습 환경

MATLAB의 진자 예제와 유사한 강화학습 환경을 PyBullet으로 구성

- **상태**: `[cos(theta), sin(theta), theta_dot]`
- **행동**: 진자 관절에 가하는 토크 (-2.0 ~ 2.0)
- **보상**: 진자가 위로 설수록, 각속도와 토크가 작을수록 높은 보상 획득.

### 1단계: URDF 파일 생성 (Code)

Jupyter Notebook의 매직 명령어 %%writefile을 사용하면 현재 디렉토리에 바로 파일을 생성할 수 있어 편리하다고 함!

In [1]:
%%writefile models/pendulum.urdf
<?xml version="1.0"?>
<robot name="pendulum">

  <link name="world"/>

  <link name="base"/>

  <link name="pole">
    <inertial>
      <mass value="1.0"/>
      <inertia ixx="0.083" ixy="0.0" ixz="0.0" iyy="0.083" iyz="0.0" izz="0.005"/>
    </inertial>
    <visual>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
      <material name="red">
          <color rgba="1 0.2 0.2 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
    </collision>
  </link>

  <joint name="world_fixed" type="fixed">
    <parent link="world"/>
    <child link="base"/>
  </joint>
  
  <joint name="hinge" type="revolute">
    <parent link="base"/>
    <child link="pole"/>
    <axis xyz="0 1 0"/>
    <limit effort="3" lower="-3.1415" upper="3.1415" velocity="15"/>
    <origin rpy="0 0 0" xyz="0 0 0.5"/>
  </joint>

</robot>

Overwriting models/pendulum.urdf


### 2단계: 라이브러리 임포트 (Code)

환경 구성과 테스트에 필요한 라이브러리들을 불러오기

In [2]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import time

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


pybullet build time: Jan 29 2025 23:16:28


### 3단계: PyBullet 진자 환경 클래스 정의

핵심이 되는 gym.Env 상속 클래스 정의

In [3]:
class PyBulletPendulumEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode='human'):
        super(PyBulletPendulumEnv, self).__init__()

        self.render_mode = render_mode
        if self.render_mode == 'human':
            # 이전에 연결된 세션이 있다면 종료 후 새로 연결
            try:
                p.disconnect()
            except p.error:
                pass
            self.client = p.connect(p.GUI)
        else:
            self.client = p.connect(p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        # URDF 파일 로드 (같은 디렉토리에 생성됨)
        self.urdf_path = "models/pendulum.urdf"
        print(f"Loading URDF from: {self.urdf_path}")
        self.pendulumId = p.loadURDF(self.urdf_path)

        # 행동 공간 (Torque)
        self.max_torque = 2.0
        self.action_space = spaces.Box(
            low=-self.max_torque,
            high=self.max_torque,
            shape=(1,),
            dtype=np.float32
        )

        # 관측 공간 [cos(theta), sin(theta), theta_dot]
        high = np.array([1.0, 1.0, 15.0], dtype=np.float32) # max velocity from URDF
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        self.step_counter = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_counter = 0

        init_angle = self.np_random.uniform(low=-np.pi, high=np.pi)
        init_velocity = self.np_random.uniform(low=-1.0, high=1.0)
        p.resetJointState(self.pendulumId, 0, targetValue=init_angle, targetVelocity=init_velocity)

        return self._get_obs(), {}

    def step(self, action):
        torque = action[0]

        p.setJointMotorControl2(
            bodyUniqueId=self.pendulumId,
            jointIndex=0,
            controlMode=p.TORQUE_CONTROL,
            force=torque
        )
        p.stepSimulation()

        obs = self._get_obs()
        reward = self._calculate_reward(obs, action)
        
        self.step_counter += 1
        truncated = self.step_counter >= 200
        terminated = False

        return obs, reward, terminated, truncated, {}

    def _get_obs(self):
        joint_state = p.getJointState(self.pendulumId, 0)
        theta, theta_dot = joint_state[0], joint_state[1]
        theta = (theta + np.pi) % (2 * np.pi) - np.pi # 각도 정규화
        return np.array([np.cos(theta), np.sin(theta), theta_dot], dtype=np.float32)

    def _calculate_reward(self, obs, action):
        cos_theta, sin_theta, theta_dot = obs
        theta = np.arctan2(sin_theta, cos_theta)
        torque = action[0]
        reward = -(theta**2 + 0.1 * theta_dot**2 + 0.001 * torque**2)
        return reward

    def render(self):
        pass

    def close(self):
        p.disconnect(self.client)

print("PyBulletPendulumEnv 클래스 정의 완료!")

PyBulletPendulumEnv 클래스 정의 완료!


### 4단계: 환경 테스트

정의된 환경을 실제로 생성하고, 무작위 행동을 주면서 어떻게 동작하는지 테스트

In [8]:
# 환경 생성
env = PyBulletPendulumEnv(render_mode='human')
obs, info = env.reset()

print("환경 테스트 시작...")

for episode in range(10): # 3번의 에피소드 테스트
    obs, info = env.reset()
    total_reward = 0
    
    for i in range(2000):
        # 무작위 행동 샘플링
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

        # GUI 환경을 위한 딜레이
        time.sleep(1./240.)
        
        if terminated or truncated:
            break
            
    print(f"에피소드 {episode + 1}: 총 보상 = {total_reward:.2f}")

print("\n환경 테스트 종료.")

# 환경 종료
env.close()
print("PyBullet 연결이 종료되었습니다.")

Loading URDF from: models/pendulum.urdf
startThreads creating 1 threads.
starting thread 0
환경 테스트 시작...
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4080 SUPER/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 580.65.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 580.65.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4080 SUPER/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data f